In [1]:
import random
import datetime
import pandas as pd
import numpy as np
from haversine import haversine
import folium
from datetime import datetime
import time
from tqdm import tqdm
import gc
import geopy.distance
import seaborn as sns
import shap
import matplotlib.pyplot as plt

## Load Data

In [2]:
df1 = pd.read_csv("data/gps_fulldata_20220512_20220819.csv", encoding='shift_jis', usecols=[0,1,2,4,6])
df2 = pd.read_csv("data/gps_20220820_20221107.csv", encoding='shift_jis', header=None, usecols=[0,1,2,4,6], names=df1.columns)


In [3]:
df = pd.concat([df1, df2],axis=0)
df.sort_values(by=['vehicleCode', 'timestamp'], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,vehicleCode,lat,lng,timestamp,distance
0,W1,35.081511,136.931423,2022-05-12T06:41:13+09:00,0.000000
1,W1,35.081820,136.931295,2022-05-12T06:49:47+09:00,36.214071
2,W1,35.081825,136.931280,2022-05-12T06:50:34+09:00,1.476148
3,W1,35.081957,136.931339,2022-05-12T06:50:39+09:00,15.601548
4,W1,35.082085,136.931459,2022-05-12T06:50:44+09:00,17.928186
...,...,...,...,...,...
33839770,座間エンジン（3時区）-2,35.501584,139.439746,2022-11-04T16:55:42+09:00,4.121708
33839771,座間エンジン（3時区）-2,35.501576,139.439728,2022-11-04T16:55:48+09:00,1.858719
33839772,座間エンジン（3時区）-2,35.501510,139.439645,2022-11-04T16:55:53+09:00,10.503742
33839773,座間エンジン（3時区）-2,35.501472,139.439588,2022-11-04T16:55:58+09:00,6.672303


In [4]:
del [[df1, df2]]
gc.collect()
df1 = pd.DataFrame()
df2 = pd.DataFrame()

In [5]:
target_df = df.copy()

## Data preprocessing

In [7]:
target_df['coord'] = target_df[['lat','lng']].apply(lambda x :tuple(x.values), axis=1)


### Define location of each timestamp
- Returns the index of the factory(1,2,3,4) if the location is in the scope of the factory
- Else returns 0

In [9]:
target_place = {}
target_place[1] = (35.50116466410529, 139.7705922117356) #"Higashi-Ogishima"
target_place[2] = (35.55769134902504, 139.66477025889324) #"K1"
target_place[3] = (35.47115639628905, 139.36695126985464) #"Atsugi"
target_place[4] = (35.520388068094086, 139.3521451687072) #"Nakatsu"

def get_gps_location(location, radi):
    for i in target_place.keys():
        if(haversine(location, target_place[i]) < radi):
            return i
    return 0

radi = 0.3 # 300m
target_df['location'] = target_df['coord'].apply(lambda x : get_gps_location(x, radi))
target_df.head()

,vehicleCode,lat,lng,timestamp,distance,coord,location
0,W1,35.081511,136.931423,2022-05-12T06:41:13+09:00,0.000000,"(35.081511, 136.931423)",0
1,W1,35.081820,136.931295,2022-05-12T06:49:47+09:00,36.214071,"(35.08182, 136.931295)",0
2,W1,35.081825,136.931280,2022-05-12T06:50:34+09:00,1.476148,"(35.081825, 136.93128)",0
3,W1,35.081957,136.931339,2022-05-12T06:50:39+09:00,15.601548,"(35.081957, 136.931339)",0
4,W1,35.082085,136.931459,2022-05-12T06:50:44+09:00,17.928186,"(35.082085, 136.931459)",0


In [11]:
# Only vehicleCode that passing 4 factories
target_pass_result= list(target_df[target_df['location']!=0]['vehicleCode'].unique())
print(target_pass_result)

['W102', 'W103', 'W104', 'W105', 'W106', 'W108', 'W109', 'W110', 'W111', 'W112', 'W113', 'W114', 'W115', 'W117', 'W118', 'W119', 'W120-1', 'W120-2', 'W40-1', 'W40-2', 'W41-1', 'W41-2', 'W42-1', 'W42-2', 'W44-1', 'W44-2', 'W44-スポット', 'W45-1', 'W45-2', 'W46', 'W46-スポット', 'W47-1', 'W47-2', 'W47-3', 'W52', 'W53', 'W54', 'W70', 'W72', 'W73', 'W74', 'W75', 'W76', 'W77', 'W78', 'W79', 'W80', 'W82-2', 'W84', 'W87-1', 'W87-2', 'W88', 'W90', 'W91', 'W92-1', 'W93-1', 'W93-2', 'W94', 'W95-1', 'W95-2', 'W97-1', 'W97-2', 'W98-1', 'W98-2', '座間エンジン（1・2・4時区）-1', '座間エンジン（1・2・4時区）-2', '座間エンジン（3時区）-1', '座間エンジン（3時区）-2']


In [12]:
target_df= target_df[target_df['vehicleCode'].isin(target_pass_result)]
target_df.reset_index(drop=True, inplace=True)
target_df

,vehicleCode,lat,lng,timestamp,distance,coord,location
0,W102,35.534938,139.651564,2022-05-12T03:49:19+09:00,40.923383,"(35.534938, 139.651564)",0
1,W102,35.534921,139.651580,2022-05-12T03:49:20+09:00,2.379729,"(35.534921, 139.65158)",0
2,W102,35.534921,139.651580,2022-05-12T03:49:24+09:00,0.000000,"(35.534921, 139.65158)",0
3,W102,35.534921,139.651580,2022-05-12T03:49:29+09:00,0.000000,"(35.534921, 139.65158)",0
4,W102,35.534976,139.651498,2022-05-12T03:49:31+09:00,9.619799,"(35.534976, 139.651498)",0
...,...,...,...,...,...,...,...
26727857,座間エンジン（3時区）-2,35.501584,139.439746,2022-11-04T16:55:42+09:00,4.121708,"(35.501584, 139.439746)",0
26727858,座間エンジン（3時区）-2,35.501576,139.439728,2022-11-04T16:55:48+09:00,1.858719,"(35.501576, 139.439728)",0
26727859,座間エンジン（3時区）-2,35.501510,139.439645,2022-11-04T16:55:53+09:00,10.503742,"(35.50151, 139.439645)",0
26727860,座間エンジン（3時区）-2,35.501472,139.439588,2022-11-04T16:55:58+09:00,6.672303,"(35.501472, 139.439588)",0


### Remove outlier

In [16]:
lng_margin = [130.8754025956282,143.49215552451497]
lat_margin = [31.939262095093653,39.08345291592023]

def check_outlier(value):
    lat, lng = value
    if ((lng<lng_margin[0]) | (lng>lng_margin[1])|(lat<lat_margin[0]) | (lat>lat_margin[1])):
        return 1
    else :
        return 0

target_df['is_outlier'] = target_df['coord'].apply(lambda x : check_outlier(x))
target_df.drop(target_df[target_df['is_outlier']==1].index, inplace = True)
#target_df.drop(columns=['coord','is_outlier'],axis=1,inplace=True)

target_df.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,vehicleCode,lat,lng,timestamp,distance,coord,location,is_outlier
0,W102,35.534938,139.651564,2022-05-12T03:49:19+09:00,40.923383,"(35.534938, 139.651564)",0,0
1,W102,35.534921,139.651580,2022-05-12T03:49:20+09:00,2.379729,"(35.534921, 139.65158)",0,0
2,W102,35.534921,139.651580,2022-05-12T03:49:24+09:00,0.000000,"(35.534921, 139.65158)",0,0
3,W102,35.534921,139.651580,2022-05-12T03:49:29+09:00,0.000000,"(35.534921, 139.65158)",0,0
4,W102,35.534976,139.651498,2022-05-12T03:49:31+09:00,9.619799,"(35.534976, 139.651498)",0,0


### Convert data type

In [18]:
import datetime
target_df['timestamp']=target_df['timestamp'].apply(lambda x: datetime.datetime.strptime(x[:-6], '%Y-%m-%dt%H:%M:%S'))

serials = target_df['vehicleCode'].unique()
for serial in serials:
    target_df.loc[target_df.vehicleCode==serial, 'del_time'] = \
    np.append(0, (target_df.loc[target_df.vehicleCode==serial, 'timestamp'][1:].reset_index(drop=True)-target_df.loc[target_df.vehicleCode==serial, 'timestamp'][:-1].reset_index(drop=True)).apply(lambda x: x.seconds).values)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
# more than 7 hours -> new course
vehicleCodes=target_df['vehicleCode'].unique()
idx = list(target_df[target_df['del_time']>7*60*60].index)

for i in vehicleCodes:
    idx.append(target_df[target_df['vehicleCode']==i].index[0])
    
idx=sorted(list(set(idx)))

target_df.loc[0, 'interval7'] = 0
target_df.loc[idx, 'interval7'] = np.arange(1, len(idx)+1)
target_df['interval7'].fillna(method='ffill', inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
Higashi_pass= list(target_df[target_df['location']==1]['interval7'].unique())
K1_pass= list(target_df[target_df['location']==2]['interval7'].unique())
Atsugi_pass= list(target_df[target_df['location']==3]['interval7'].unique())
Nakatsu_pass= list(target_df[target_df['location']==4]['interval7'].unique())

print("Higashi_count = {}\nK1_count = {}\nAtsugi_count = {}\nNakatsu_count = {}".format(len(Higashi_pass),len(K1_pass),len(Atsugi_pass),len(Nakatsu_pass)))

Higashi_count = 1459
K1_count = 4851
Atsugi_count = 1874
Nakatsu_count = 1612


In [27]:
#Only drives that passing among 4 factories
Higashi_data_index = []
K1_data_index=[]
Atsugi_data_index = []
Nakatsu_data_index =[]

for drive in Higashi_pass:
    temp =target_df[target_df['interval7']==drive]
    start = temp.index[0]
    temp = temp[temp['location']==1]
    if(len(temp)<10):
        continue 
    if(temp.iloc[9].location==1):
        end = temp.iloc[9].name
        Higashi_data_index.extend([x for x in range(start,end)])
        
for drive in K1_pass:
    temp =target_df[target_df['interval7']==drive]
    start = temp.index[0]
    temp = temp[temp['location']==1]
    if(len(temp)<10):
        continue 
    if(temp.iloc[9].location==1):
        end = temp.iloc[9].name
        K1_data_index.extend([x for x in range(start,end)])
        
for drive in Atsugi_pass:
    temp =target_df[target_df['interval7']==drive]
    start = temp.index[0]
    temp = temp[temp['location']==1]
    if(len(temp)<10):
        continue 
    if(temp.iloc[9].location==1):
        end = temp.iloc[9].name
        Atsugi_data_index.extend([x for x in range(start,end)])

for drive in Nakatsu_pass:
    temp =target_df[target_df['interval7']==drive]
    start = temp.index[0]
    temp = temp[temp['location']==1]
    if(len(temp)<10):
        continue 
    if(temp.iloc[9].location==1):
        end = temp.iloc[9].name
        Nakatsu_data_index.extend([x for x in range(start,end)])

In [28]:
print(len(Higashi_data_index))
print(len(K1_data_index))
print(len(Atsugi_data_index))
print(len(Nakatsu_data_index))

3408781
1995286
814493
700930


In [30]:
target_df['weekday'] = target_df['timestamp'].dt.weekday
target_df['velocity'] = target_df['distance']/target_df['del_time']
target_df


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,level_0,index,vehicleCode,lat,lng,timestamp,distance,coord,location,is_outlier,del_time,interval7,weekday,velocity
0,0,0,W102,35.534938,139.651564,2022-05-12 03:49:19,40.923383,"(35.534938, 139.651564)",0,0,0.0,1.0,3,inf
1,1,1,W102,35.534921,139.651580,2022-05-12 03:49:20,2.379729,"(35.534921, 139.65158)",0,0,1.0,1.0,3,2.379729
2,2,2,W102,35.534921,139.651580,2022-05-12 03:49:24,0.000000,"(35.534921, 139.65158)",0,0,4.0,1.0,3,0.000000
3,3,3,W102,35.534921,139.651580,2022-05-12 03:49:29,0.000000,"(35.534921, 139.65158)",0,0,5.0,1.0,3,0.000000
4,4,4,W102,35.534976,139.651498,2022-05-12 03:49:31,9.619799,"(35.534976, 139.651498)",0,0,2.0,1.0,3,4.809900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26723650,26723650,26727857,座間エンジン（3時区）-2,35.501584,139.439746,2022-11-04 16:55:42,4.121708,"(35.501584, 139.439746)",0,0,5.0,6818.0,4,0.824342
26723651,26723651,26727858,座間エンジン（3時区）-2,35.501576,139.439728,2022-11-04 16:55:48,1.858719,"(35.501576, 139.439728)",0,0,6.0,6818.0,4,0.309787
26723652,26723652,26727859,座間エンジン（3時区）-2,35.501510,139.439645,2022-11-04 16:55:53,10.503742,"(35.50151, 139.439645)",0,0,5.0,6818.0,4,2.100748
26723653,26723653,26727860,座間エンジン（3時区）-2,35.501472,139.439588,2022-11-04 16:55:58,6.672303,"(35.501472, 139.439588)",0,0,5.0,6818.0,4,1.334461


In [36]:
target_df['five_sec'] = target_df['timestamp'].dt.round('5s').dt.time
target_df['five_sec'] = target_df['five_sec'].apply(lambda x: x.strftime("%H:%M:%S"))
target_df.loc[target_df['velocity']==np.inf, 'velocity'] = 0

a = pd.DataFrame(target_df.groupby('five_sec')['velocity'].mean())
a.rename(columns = {'velocity':'velocity_mean_5sec'},inplace=True)
b = pd.DataFrame(target_df.groupby('five_sec')['velocity'].std())
b.rename(columns = {'velocity':'velocity_std_5sec'},inplace=True)
df1=pd.concat([a,b],axis=1).reset_index()

c = pd.DataFrame(target_df.groupby('weekday')['velocity'].mean())
c.rename(columns = {'velocity':'velocity_mean_weekday'},inplace=True)
d = pd.DataFrame(target_df.groupby('weekday')['velocity'].std())
d.rename(columns = {'velocity':'velocity_std_weekday'},inplace=True)
df2=pd.concat([c,d],axis=1).reset_index()

target_df = pd.merge(target_df, df1, on='five_sec', how='left')
target_df = pd.merge(target_df, df2, on='weekday', how='left')

# vehicleCode별로 코스의 총거리 평균
e = pd.DataFrame(target_df.groupby(["vehicleCode",'interval7']).distance.sum().reset_index().groupby('vehicleCode').distance.mean())
e.rename(columns = {'distance':'total_distance_mean_vehicle'},inplace=True)
# vehicleCode별로 코스의 총시간 평균
f = pd.DataFrame(target_df.groupby(["vehicleCode","interval7"]).del_time.sum().reset_index().groupby('vehicleCode').del_time.mean())
f.rename(columns = {'del_time':'total_time_mean_vehicle'},inplace=True)
df3=pd.concat([e,f],axis=1).reset_index()

target_df = pd.merge(target_df, df3, on='vehicleCode', how='left')
target_df


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,level_0,index,vehicleCode,lat,lng,timestamp,distance,coord,location,is_outlier,...,interval7,weekday,velocity,five_sec,velocity_mean_5sec,velocity_std_5sec,velocity_mean_weekday,velocity_std_weekday,total_distance_mean_vehicle,total_time_mean_vehicle
0,0,0,W102,35.534938,139.651564,2022-05-12 03:49:19,40.923383,"(35.534938, 139.651564)",0,0,...,1.0,3,0.000000,03:49:20,11.706075,8.546284,14.280946,504.669762,113718.354540,86089.450382
1,1,1,W102,35.534921,139.651580,2022-05-12 03:49:20,2.379729,"(35.534921, 139.65158)",0,0,...,1.0,3,2.379729,03:49:20,11.706075,8.546284,14.280946,504.669762,113718.354540,86089.450382
2,2,2,W102,35.534921,139.651580,2022-05-12 03:49:24,0.000000,"(35.534921, 139.65158)",0,0,...,1.0,3,0.000000,03:49:25,11.671444,9.312073,14.280946,504.669762,113718.354540,86089.450382
3,3,3,W102,35.534921,139.651580,2022-05-12 03:49:29,0.000000,"(35.534921, 139.65158)",0,0,...,1.0,3,0.000000,03:49:30,13.289898,44.053619,14.280946,504.669762,113718.354540,86089.450382
4,4,4,W102,35.534976,139.651498,2022-05-12 03:49:31,9.619799,"(35.534976, 139.651498)",0,0,...,1.0,3,4.809900,03:49:30,13.289898,44.053619,14.280946,504.669762,113718.354540,86089.450382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26723650,26723650,26727857,座間エンジン（3時区）-2,35.501584,139.439746,2022-11-04 16:55:42,4.121708,"(35.501584, 139.439746)",0,0,...,6818.0,4,0.824342,16:55:40,15.487454,59.592917,12.733705,50.916114,122904.112538,86085.165138
26723651,26723651,26727858,座間エンジン（3時区）-2,35.501576,139.439728,2022-11-04 16:55:48,1.858719,"(35.501576, 139.439728)",0,0,...,6818.0,4,0.309787,16:55:50,18.590636,123.697314,12.733705,50.916114,122904.112538,86085.165138
26723652,26723652,26727859,座間エンジン（3時区）-2,35.501510,139.439645,2022-11-04 16:55:53,10.503742,"(35.50151, 139.439645)",0,0,...,6818.0,4,2.100748,16:55:55,16.172334,48.119371,12.733705,50.916114,122904.112538,86085.165138
26723653,26723653,26727860,座間エンジン（3時区）-2,35.501472,139.439588,2022-11-04 16:55:58,6.672303,"(35.501472, 139.439588)",0,0,...,6818.0,4,1.334461,16:56:00,16.108201,59.934680,12.733705,50.916114,122904.112538,86085.165138


# 

In [78]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
import pickle
from tqdm import tqdm

seed = 1422
def feature_engineering(df,i):
    total_distance = df.groupby('interval7').distance.sum()
    df['total_distance/drive'] = df['interval7'].apply(lambda x:total_distance[x])
    df['cum_sum_distance'] = df.groupby("interval7").distance.cumsum()
    
    ata_dic = df.groupby('interval7')['timestamp'].max()
    df['ata'] = df['interval7'].apply(lambda x : ata_dic[x])
    
    df['left_time'] = df['ata']- df['timestamp']
    df['left_time'] = df['left_time'].apply(lambda x : x.total_seconds()//60)
    df['distance_btw_target'] = df['coord'].apply(lambda x : haversine(x,target_place[i]))

    return df

using_cols = ['lat','lng','distance','cum_sum_distance','velocity','distance_btw_target','weekday','velocity_mean_5sec','velocity_std_5sec','velocity_mean_weekday','velocity_std_weekday','total_distance_mean_vehicle','total_time_mean_vehicle','left_time']
legend ={1: 'Higashi', 2: 'K1', 3 : 'Atsugi',4:'Nakatsu'}

for i,target in enumerate([Higashi_data_index,K1_data_index,Atsugi_data_index,Nakatsu_data_index]):
    data = target_df.loc[target]
    data = feature_engineering(data,i+1)
    for j,r in tqdm(enumerate(data['vehicleCode'].unique())):
        file_name = 'models/{}_{}_model.model'.format(legend[i+1],r)
        
        
        dff = data[data['vehicleCode']==r]
        dff = dff[using_cols]
        y= dff['left_time']
        X = dff.drop(['left_time'],axis=1)
        x_train, x_test , y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state= 1422) 

        model= CatBoostRegressor(
                          eval_metric = 'MAE',
                          cat_features = ['weekday'],
                          random_state=seed,
                         verbose = False
                      )

        model.fit(x_train , y_train,early_stopping_rounds =50)
        
        pred = model.predict(x_test)
        mse = mean_squared_error(y_test , pred)
        mae = mean_absolute_error(y_test, pred)
        r2 = r2_score(y_test, pred)
        print(f'{file_name} : rmse : {np.sqrt(mse)} mae : {mae}  r2 : {r2}')
        pickle.dump(model, open(file_name, 'wb'))
        

1it [00:01,  1.37s/it]

models/Higashi_W103_model.model : rmse : 4.713199977224574 mae : 2.6553905508165854  r2 : 0.9999527350075801


2it [00:19, 11.24s/it]

models/Higashi_W106_model.model : rmse : 8.700080089632147 mae : 4.5593386203766855  r2 : 0.9975178000645533


3it [01:26, 36.88s/it]

...

models/Nakatsu_W47-2_model.model : rmse : 8.34342937478785 mae : 3.7020728438943444  r2 : 0.9999747581702811


10it [02:58, 12.82s/it]

models/Nakatsu_W80_model.model : rmse : 4.663830406247174 mae : 2.8648098013056784  r2 : 0.998975280665863


11it [03:00, 16.43s/it]

models/Nakatsu_W90_model.model : rmse : 4.539483137344271 mae : 2.97837399453098  r2 : 0.9990312631849025
